<a href="https://colab.research.google.com/github/Nanashi-bot/kaggle/blob/main/kaggles5e1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import pandas as pd
import numpy as np
import time

In [5]:
df = pd.read_csv("train.csv")

In [6]:
df.head()

,id,date,country,store,product,num_sold
0,0,2010-01-01,Canada,Discount Stickers,Holographic Goose,NaN
1,1,2010-01-01,Canada,Discount Stickers,Kaggle,973.0
2,2,2010-01-01,Canada,Discount Stickers,Kaggle Tiers,906.0
3,3,2010-01-01,Canada,Discount Stickers,Kerneler,423.0
4,4,2010-01-01,Canada,Discount Stickers,Kerneler Dark Mode,491.0


In [7]:
df = df.dropna()

In [8]:
df.isna().sum()

,0
id,0
date,0
country,0
store,0
product,0
num_sold,0


In [9]:
print(df['country'].unique())
print(df['store'].unique())
print(df['product'].unique())

['Canada' 'Finland' 'Italy' 'Kenya' 'Norway' 'Singapore']
['Discount Stickers' 'Stickers for Less' 'Premium Sticker Mart']
['Kaggle' 'Kaggle Tiers' 'Kerneler' 'Kerneler Dark Mode'
 'Holographic Goose']


In [10]:
df = pd.get_dummies(df, columns=['country', 'store', 'product'])

In [11]:
df['date'] = pd.to_datetime(df['date'])
df['Year'] = df['date'].dt.year
df['Month'] = df['date'].dt.month
# df['Day'] = df['date'].dt.day
df['Day_Sin'] = np.sin(2 * np.pi * df['date'].dt.dayofweek / 7)
df['Day_Cos'] = np.cos(2 * np.pi * df['date'].dt.dayofweek / 7)
df['Quarter'] = df['date'].dt.quarter  # 1, 2, 3, or 4
# df['Week_of_Year'] = df['date'].dt.isocalendar().week  # Week number
df['Is_Weekend'] = df['date'].dt.weekday >= 5  # True if Saturday/Sunday

In [12]:
df = df.drop(['id', 'date'], axis = 1)

In [13]:
df['Year'].unique()

array([2010, 2011, 2012, 2013, 2014], dtype=int32)

In [14]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
df[['Year', 'Month', 'Quarter']] = scaler.fit_transform(df[['Year', 'Month', 'Quarter']])

In [15]:
# df[:1].iloc[0]

In [16]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error

X = df.drop(columns=['num_sold'])
y = df['num_sold']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [16]:
# LINEAR REGRESSION
from sklearn.linear_model import LinearRegression

model = LinearRegression()
model.fit(X_train, y_train)  # Train the model

y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)

print(f"Root Mean Squared Error (RMSE): {rmse:.2f}")

Root Mean Squared Error (RMSE): 311.01
R² Score: 0.8105


In [17]:
# ELASTIC NET
from sklearn.linear_model import ElasticNet

elastic = ElasticNet(alpha=0.01, l1_ratio=0.5)  # l1_ratio=0 is Ridge, 1 is Lasso
elastic.fit(X_train, y_train)
y_pred_elastic = elastic.predict(X_test)


mse = mean_squared_error(y_test, y_pred_elastic)
rmse = np.sqrt(mse)

print(f"Root Mean Squared Error (RMSE): {rmse:.2f}")

Root Mean Squared Error (RMSE): 311.71
R² Score: 0.8097


In [15]:
# Polynomial Regression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline

poly = make_pipeline(PolynomialFeatures(degree=2), LinearRegression())
poly.fit(X_train, y_train)
y_pred_poly = poly.predict(X_test)

mse = mean_squared_error(y_test, y_pred_poly)
rmse = np.sqrt(mse)

print(f"Root Mean Squared Error (RMSE): {rmse:.2f}")

NameError: name 'LinearRegression' is not defined

In [20]:
# Polynomial Regression with degree 3
poly = make_pipeline(PolynomialFeatures(degree=3), LinearRegression())
poly.fit(X_train, y_train)
y_pred_poly = poly.predict(X_test)

mse = mean_squared_error(y_test, y_pred_poly)
rmse = np.sqrt(mse)

print(f"Root Mean Squared Error (RMSE): {rmse:.2f}")

Root Mean Squared Error (RMSE): 90.34
R² Score: 0.9840


In [ ]:
# LIGHTGBM

import lightgbm as lgb
import pandas as pd
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.metrics import mean_squared_error

# Create LightGBM dataset
train_data = lgb.Dataset(X_train, label=y_train)
test_data = lgb.Dataset(X_test, label=y_test, reference=train_data)

param_dist = {
    'num_leaves': [31, 50, 100, 150],
    'max_depth': [-1, 5, 10, 15],
    'learning_rate': [0.01, 0.05, 0.1, 0.2],
    'n_estimators': [50, 100, 150, 200],
    'subsample': [0.5, 0.7, 0.9],
    'colsample_bytree': [0.5, 0.7, 1],
    'lambda_l1': [0, 0.1, 0.5],
    'lambda_l2': [0, 0.1, 0.5]
}

model = lgb.LGBMRegressor()

random_search = RandomizedSearchCV(model, param_distributions=param_dist, n_iter=50, cv=5, scoring='neg_mean_squared_error', random_state=42, n_jobs=-1)
# random_search.fit(X_train, y_train)
random_search.fit(X_train, y_train, eval_set=[(X_test, y_test)], eval_metric='rmse')
# random_search.fit(X, y)

print(f'Best hyperparameters: {random_search.best_params_}')
best_model_lgbm = random_search.best_estimator_

y_pred_lgbm = best_model_lgbm.predict(X_test)

rmse = np.sqrt(mean_squared_error(y_test, y_pred_lgbm))
print(f'Root Mean Squared Error (RMSE): {rmse}')

In [3]:
!pip uninstall -y scikit-learn
!pip install scikit-learn==1.3.1

Found existing installation: scikit-learn 1.6.1
Uninstalling scikit-learn-1.6.1:
  Successfully uninstalled scikit-learn-1.6.1
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.9/10.9 MB 88.7 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
imbalanced-learn 0.13.0 requires scikit-learn<2,>=1.3.2, but you have scikit-learn 1.3.1 which is incompatible.


In [20]:
import xgboost as xgb
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.metrics import mean_squared_error

param_dist = {
    'max_depth': [3, 5, 7, 10],
    'learning_rate': [0.01, 0.05, 0.1, 0.2],
    'n_estimators': [50, 100, 150, 200],
    'subsample': [0.5, 0.7, 0.9],
    'colsample_bytree': [0.5, 0.7, 1],
    'alpha': [0, 0.1, 0.5],  # L1 regularization
    'lambda': [0, 0.1, 0.5],  # L2 regularization
    'gamma': [0, 0.1, 0.5],  # Minimum loss reduction
}

model = xgb.XGBRegressor()

random_search = RandomizedSearchCV(model, param_distributions=param_dist, n_iter=50, cv=5, scoring='neg_mean_squared_error', random_state=42, n_jobs=-1)
# random_search.fit(X_train, y_train)
random_search.fit(X_train, y_train, eval_set=[(X_test, y_test)], eval_metric='rmse')
# random_search.fit(X, y)

print(f'Best hyperparameters: {random_search.best_params_}')
best_model_xgb = random_search.best_estimator_

y_pred_xgb = best_model_xgb.predict(X_test)

rmse = np.sqrt(mean_squared_error(y_test, y_pred_xgb))
print(f'Root Mean Squared Error (RMSE): {rmse}')

Best hyperparameters: {'subsample': 0.7, 'n_estimators': 150, 'max_depth': 7, 'learning_rate': 0.1, 'lambda': 0.1, 'gamma': 0, 'colsample_bytree': 0.7, 'alpha': 0}
Root Mean Squared Error (RMSE): 73.3210202437254


In [26]:
from sklearn.linear_model import LinearRegression
# Stack predictions from both models
y_pred_xgb = best_model_xgb.predict(X_train)
y_pred_lgbm = best_model_xgb.predict(X_train)
stacked_train = np.column_stack((y_pred_xgb, y_pred_lgbm))

# Train a meta-model (e.g., Linear Regression)
meta_model = LinearRegression()
meta_model.fit(stacked_train, y_train)

LinearRegression()

In [27]:
test = pd.read_csv("test.csv")

In [28]:
test.head()

,id,date,country,store,product
0,230130,2017-01-01,Canada,Discount Stickers,Holographic Goose
1,230131,2017-01-01,Canada,Discount Stickers,Kaggle
2,230132,2017-01-01,Canada,Discount Stickers,Kaggle Tiers
3,230133,2017-01-01,Canada,Discount Stickers,Kerneler
4,230134,2017-01-01,Canada,Discount Stickers,Kerneler Dark Mode


In [29]:
test.isna().sum().sum()

0

In [30]:
test = pd.get_dummies(test, columns=['country', 'store', 'product'])

In [31]:
test['date'] = pd.to_datetime(test['date'])
test['Year'] = test['date'].dt.year
test['Month'] = test['date'].dt.month
test['Day_Sin'] = np.sin(2 * np.pi * test['date'].dt.dayofweek / 7)
test['Day_Cos'] = np.cos(2 * np.pi * test['date'].dt.dayofweek / 7)
test['Quarter'] = test['date'].dt.quarter  # 1, 2, 3, or 4
# df['Week_of_Year'] = df['date'].dt.isocalendar().week  # Week number
test['Is_Weekend'] = test['date'].dt.weekday >= 5  # True if Saturday/Sunday

In [32]:
test = test.drop(['date'], axis = 1)

In [33]:
test[['Year', 'Month', 'Quarter']] = scaler.fit_transform(test[['Year', 'Month', 'Quarter']])

In [34]:
test.head()

,id,country_Canada,country_Finland,country_Italy,country_Kenya,country_Norway,country_Singapore,store_Discount Stickers,store_Premium Sticker Mart,store_Stickers for Less,...,product_Kaggle,product_Kaggle Tiers,product_Kerneler,product_Kerneler Dark Mode,Year,Month,Day_Sin,Day_Cos,Quarter,Is_Weekend
0,230130,True,False,False,False,False,False,True,False,False,...,False,False,False,False,-1.224745,-1.602745,-0.781831,0.62349,-1.351749,True
1,230131,True,False,False,False,False,False,True,False,False,...,True,False,False,False,-1.224745,-1.602745,-0.781831,0.62349,-1.351749,True
2,230132,True,False,False,False,False,False,True,False,False,...,False,True,False,False,-1.224745,-1.602745,-0.781831,0.62349,-1.351749,True
3,230133,True,False,False,False,False,False,True,False,False,...,False,False,True,False,-1.224745,-1.602745,-0.781831,0.62349,-1.351749,True
4,230134,True,False,False,False,False,False,True,False,False,...,False,False,False,True,-1.224745,-1.602745,-0.781831,0.62349,-1.351749,True


In [35]:
ids = test['id']
test = test.drop(['id'], axis = 1)

In [36]:
y_final_xgb = best_model_xgb.predict(test)
y_final_lgbm = best_model_lgbm.predict(test)

[LightGBM] [Warning] lambda_l2 is set=0.1, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.1
[LightGBM] [Warning] lambda_l1 is set=0.5, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.5


In [33]:
# Assign weights based on RMSE performance (lower RMSE means higher weight)
weight_lgb = 0.6
weight_xgb = 0.4

combined_preds_weighted = (weight_lgb * y_final_lgbm) + (weight_xgb * y_final_xgb)

In [37]:
# Stack predictions from both models
stacked_train = np.column_stack((y_final_xgb, y_final_lgbm))

xgb_preds_test = best_model_xgb.predict(test)
lgbm_preds_test = best_model_lgbm.predict(test)

# Stack test predictions from both models
stacked_test = np.column_stack((xgb_preds_test, lgbm_preds_test))

# Make final predictions using the meta-model
final_preds = meta_model.predict(stacked_test)

[LightGBM] [Warning] lambda_l2 is set=0.1, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.1
[LightGBM] [Warning] lambda_l1 is set=0.5, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.5


In [38]:
y_final = pd.Series(final_preds)
final_df = pd.concat([ids, y_final], axis=1)
final_df.columns = ['id', 'num_sold']

In [39]:
final_df.head()

,id,num_sold
0,230130,153.401121
1,230131,802.234845
2,230132,714.452384
3,230133,349.668574
4,230134,446.134222


In [40]:
final_df.to_csv('result5.csv', index=False)